### Generation of predictions using BioTransformer
- Created by: Louis Groff
- PIs: Imran Shah and Grace Patlewicz
- Last modified: 4 March 2024
- Changes made: Additional note on implementation added from the SI

A working copy of BioTransformer is needed. Assuming the input directory contains the BioTransformer Executable Java Archive (JAR) the full Command Line Interface (CLI) input command would be: java -jar BioTransformer3.0_20220615.jar -k pred -q "ecbased:1;cyp450:2;phaseII:1" -cm 1 -ismi "<in.hcd_smiles>" -ocsv ".\tmpfiles\btrans_out_<in.dtxsid>_randomfilename string>.csv" The randomly suffixed output files generated via the “tempfile” package in Python can either be kept or discarded after data processing is completed. 

For the validation datasets, the models used were either “cyp450” for single-step phase I metabolism (Table S2 in the supplemental information), or any mixture of “cyp450”, “ecbased” and “phaseII” that terminated in phase II, whether individual runs or sequential. The CLI command for the highest performing single-step human model (no gut) was:
java -jar BioTransformer3.0_20220615.jar -k pred -q "ecbased:1;cyp450:1;phaseII:1" -cm 1 -ismi "<in:smiles>" -ocsv ".\tmpfiles\btrans_out_<in.dtxsid>_randomfile name string>.csv"

# 1. metsim_hcd_out():
     Wrapped within the Toolbox WebAPI or BioTransformer calling functions. (Does not require VPN)
     input: SMILES string (required), optional are the DTXSID (required for CCD searching), CASRN, and Chemical Names.
     action: queries the EPA Hazard Comparison Dashboard (HCD) for JSON output data for a given chemical. If output data exists, relevant chemical/structural identifiers are supplemented to existing metsim outputs.
     output: If they exist - DTXSID, Canonical SMILES (hcd_smiles key), CASRN, and InChIKey (via HCD, or RDKit if not in HCD).
# 2. metsim_metadata_full():
    used to save time between running BioTransformer and gathering metabolite metadata. Since BioTransformer suffers more from the "combinatorial explosion" issues of having so many metabolites it produces, which is computationally taxing, we run BioTransformer and save the SMILES and generational tracking from it in our dictionary format as a list of dictionaries, and then later use metsim_metadata_full to fill in the metadata for this list of dictionaries.
# 3. btrans_metsim_subprocess():
    input: SMILES, List of Models, List of Cycles, Delete Tempfile (True/False)
    action: Simulates human metabolism (Default: 2 cycles Phase I/"cyp450", 1 cycle Phase II/"phaseII") using BioTransformer 3.0 through Java in the command prompt, producing output data in temporary files that can be kept or deleted. Recursively searches through output CSV to process data into a standardized dictionary output.
    output: Tuple with dummy index (for parallel processing), Dictionary of precursor and successor SMILES, CASRN, DTXSID, InChIKey as supplemented by HCD (or RDKit for InChIKey), and filename (if del_tmp = False).

In [2]:
import os
import subprocess
import numpy as np
import pandas as pd
import urllib.request, urllib.parse, json
import time
import tempfile
from rdkit import Chem
import datetime
#import multiprocess as mp
from itertools import compress
import pprint

In [3]:
#Function 1: HCD Queries
def metsim_hcd_out(smiles = None, 
                   casrn = None,
                   dtxsid = None,
                   inchikey = None,
                   chem_name = None,
                   likely = None):
    """
    Query function for the Cheminformatics Modules Standardizer API, formerly wrapped within the Hazard Comparison Dashboard (HCD) API. 
    Used to convert an input SMILES string into QSAR-Ready SMILES. Returns InChIKey structural identifier as well,
    along with any other chemical identifer metadata if available, and not already given as inputs (e.g., CASRN, DTXSID, Chemical Name).
    
    If SMILES is not known, but DTXSID is known, can instead query on DTXSID to obtain Daylight SMILES from the Comptox Chemicals Dashboard API (CCD API),
    and subsequently query the Standardizer API using the SMILES obtained from the CCD API.
    
    Required Inputs:
    smiles: Daylight SMILES string
    or
    dtxsid: DSSTox Substance Identifier
    
    Optional Inputs:
    chem_name: Chemical name, whether trade name or IUPAC
    casrn: Chemical Abstracts Services Registry Number
    inchikey: International Chemical Identifier Key (InChIKey)
    likely: If MetSim predictions are obtained from the Chemical Transformation Simulator, can optionally keep the transformation "likelihood" parameter
    
    Returns:
    out_dict: Output dictionary containing all available output data for the given chemical, using the input parameter names as dictionary keys.
    Includes "hcd_smiles" as output dictionary key containing QSAR-Ready version of the input SMILES.
    
    Examples:
    
    SMILES given as sole input:
    input:    
    test_dict = metsim_hcd_out(smiles = "OCCOCCO")
    
    output:
    Attempting query of Cheminformatics Modules Standardizer with SMILES: OCCOCCO...
    Query succeeded.
    test_dict
    {'smiles': 'OCCOCCO',
     'casrn': '111-46-6',
     'hcd_smiles': 'OCCOCCO',
     'inchikey': 'MTHSVFCYNBDYFN-UHFFFAOYNA-N',
     'dtxsid': 'DTXSID8020462',
     'chem_name': 'Diethylene glycol',
     'likelihood': None}
    
    DTXSID given as sole input:
    
    input:    
    test_dict = metsim_hcd_out(dtxsid = "DTXSID4020402")
    
    output:
    Attempting query of Comptox Chemicals Dashboard with DTXSID: DTXSID4020402...
    Query succeeded.
    No SMILES given. Using CCD output SMILES.
    Attempting query of Cheminformatics Modules Standardizer with SMILES: CC1=C(N)C=C(N)C=C1...
    Query succeeded.
    test_dict
    {'smiles': 'CC1=C(N)C=C(N)C=C1',
     'casrn': '95-80-7',
     'hcd_smiles': 'CC1C=CC(N)=CC=1N',
     'inchikey': 'VOZKAJLKRJDJLL-UHFFFAOYNA-N',
     'dtxsid': 'DTXSID4020402',
     'chem_name': '2,4-Diaminotoluene',
     'likelihood': None}
     
     Empty inputs:
     input:
     test_dict = metsim_hcd_out(smiles = None, dtxsid = None)
     
     output:
     test_dict
     {'smiles': None,
     'casrn': None,
     'hcd_smiles': None,
     'inchikey': None,
     'dtxsid': None,
     'chem_name': None,
     'likelihood': None}
    """
    
    ccd_out = []
    if dtxsid != None and smiles == None:
        #get metadata from Comptox Chemicals Dashboard for a given DTXSID (No structure searching atm).
        ccd_url = 'https://comptox.epa.gov/dashboard-api/ccdapp2/chemical-detail/search/by-dsstoxsid?id='+dtxsid
        ccd_success = 0
        try_count = 0
        while ccd_success == 0 and try_count < 3:
            try:
                print('Attempting query of Comptox Chemicals Dashboard with DTXSID: '+dtxsid+'...')
                ccd_out = json.loads(urllib.request.urlopen(ccd_url).read().decode())
                
                ccd_success = 1
                try_count+=1
            except:
                #Given that this occasionally fails randomly due to timeout errors, 
                #but then works again later, try again after a 1 second pause.
                #Should work on second attempt.
                print('URL Error Occurred, reattempting CCD query in 0.5 seconds.')
                time.sleep(0.5)
                try_count+=1
            print('Query succeeded.')
    if smiles != None or len(ccd_out) > 0:
        if smiles != None:
            smiles_url = urllib.parse.quote_plus(smiles) #URL encode SMILES string.
        elif len(ccd_out) > 0 and ccd_out['smiles'] != None:
            print('No SMILES given. Using CCD output SMILES.')
            smiles = ccd_out['smiles']
            smiles_url = urllib.parse.quote_plus(smiles) #URL enconde CCD smiles string.
        else:
            print('No SMILES given, and no SMILES available from CCD output.')
            smiles_url = None
        base_url = "https://hcd.rtpnc.epa.gov/api/stdizer?workflow=qsar-ready&smiles=" #Production environment (current, no VPN needed)
        # base_url = "https://hazard-dev.sciencedataexperts.com/api/stdizer?workflow=qsar-ready&smiles=" #Dev environment (no VPN needed)
        # base_url = "https://hazard.sciencedataexperts.com/api/stdizer?workflow=qsar-ready&smiles=" #Production environment (VPN needed)
        if smiles_url != None:
            hcd_url = base_url+smiles_url
            hcd_success = 0
            try_count = 0
            hcd_out = []
            while hcd_success == 0 and try_count < 3:
                try:
                    print('Attempting query of Cheminformatics Modules Standardizer with SMILES: '+smiles+'...')
                    time.sleep(0.5)
                    hcd_out = json.loads(urllib.request.urlopen(hcd_url).read().decode())
                    print('Query succeeded.')
                    hcd_success = 1
                    try_count+=1
                except:
                    #Given that this occasionally fails randomly due to timeout errors, 
                    #but then works again later, try again after a 1 second pause.
                    #Should work on second attempt.
                    print('URL Error Occurred, reattempting Cheminformatics Modules query in 0.5 seconds.')
                    time.sleep(0.5)
                    try_count+=1
            if len(hcd_out) > 0:
                out_dict = {'smiles': smiles, 
                            'casrn': casrn,
                            'hcd_smiles': hcd_out[0]['smiles'],
                            'inchikey': hcd_out[0]['inchiKey'],
                            'dtxsid': dtxsid,
                            'chem_name': chem_name,
                            'likelihood': likely}
                if out_dict['dtxsid'] == None:
                    if 'DTXSID' in hcd_out[0]['id']:
                        out_dict['dtxsid'] = hcd_out[0]['id']
                    elif len(ccd_out) > 0 and ccd_out['dsstoxSubstanceId'] != None:
                        out_dict['dtxsid'] = ccd_out['dsstoxSubstanceId']
                if out_dict['casrn'] == None:
                    if 'casrn' in hcd_out[0].keys():
                        out_dict['casrn'] = hcd_out[0]['casrn']
                    elif len(ccd_out) > 0 and ccd_out['casrn'] != None:
                        out_dict['casrn'] = ccd_out['casrn']
                if out_dict['chem_name'] == None:
                    if len(ccd_out) > 0 and ccd_out['preferredName'] != None:
                        out_dict['chem_name'] = ccd_out['preferredName']
                    elif 'name' in hcd_out[0].keys():
                        out_dict['chem_name'] = hcd_out[0]['name']
                if out_dict['inchikey'] == None and len(ccd_out) > 0:
                    if ccd_out['inchiKey'] != None:
                        out_dict['inchikey'] = ccd_out['inchiKey'] 
            else:
                out_dict = {'smiles': smiles,
                            'casrn': casrn,
                            'hcd_smiles': None,
                            'inchikey': None,
                            'dtxsid': dtxsid,
                            'chem_name': chem_name,
                            'likelihood': likely
                           }
                #HCD Returns empty list. Try to supplement with metadata from RDKit.
                try:
                    smiles_mol = Chem.MolFromSmiles(smiles)
                    out_dict['inchikey'] = Chem.inchi.MolToInchiKey(smiles_mol)
                except:
                    #Rarely, BioTransformer makes a bad SMILES string for a metabolite, and RDKit can't convert it to an InChIKey. Store None
                    print('RDKit failed to generate an inchikey for SMILES: '+smiles)
                    out_dict['inchikey'] = None
        else:
            out_dict = {'smiles': smiles,
                        'casrn': casrn,
                        'hcd_smiles': None,
                        'inchikey': None,
                        'dtxsid': dtxsid,
                        'chem_name': chem_name,
                        'likelihood': likely
                       }
    else:
        out_dict = {'smiles': smiles,
                    'casrn': casrn,
                    'hcd_smiles': None,
                    'inchikey': None,
                    'dtxsid': dtxsid,
                    'chem_name': chem_name,
                    'likelihood': likely
                   }
    return out_dict

In [4]:
def metsim_metadata_full(metsim_out = [], fnam = None, metsim_cache = None):
    
    if len(metsim_out) > 0:
        if metsim_cache != None:
            #Supplement metadata via serial HCD query through individual input chemicals, precursors, successors/metabolites for a full metsim dataset
            for i in range(len(metsim_out)): # i = number of input chemicals
                if metsim_out[i]['input']['inchikey'] != None:
                        continue
                if metsim_out[i]['input']['smiles'] not in [cache_item['smiles'] for cache_item in metsim_cache]:
                    metsim_out[i]['input'] = metsim_hcd_out(smiles = metsim_out[i]['input']['smiles'],
                                                            casrn = metsim_out[i]['input']['casrn'],
                                                            dtxsid = metsim_out[i]['input']['dtxsid'],
                                                            chem_name = metsim_out[i]['input']['chem_name'])
                    metsim_cache.append(metsim_out[i]['input'])
                    print('Input query added to metadata cache...')
                else:
                    print('Input SMILES found in cached results. Inserting into dictionary...')
                    metsim_out[i]['input'] = metsim_cache[[idx for idx in range(len(metsim_cache)) if metsim_cache[idx]['smiles'] == metsim_out[i]['input']['smiles']][0]]
                for j in range(len(metsim_out[i]['output'])): # j = number of unique precursors
                    if 'likelihood' in list(metsim_out[i]['output'][j]['precursor'].keys()):
                        if metsim_out[i]['output'][j]['precursor']['smiles'] not in [cache_item['smiles'] for cache_item in metsim_cache]:
                            metsim_out[i]['output'][j]['precursor'] = metsim_hcd_out(smiles = metsim_out[i]['output'][j]['precursor']['smiles'],
                                                                                     casrn = metsim_out[i]['output'][j]['precursor']['casrn'],
                                                                                     dtxsid = metsim_out[i]['output'][j]['precursor']['dtxsid'],
                                                                                     chem_name = metsim_out[i]['output'][j]['precursor']['chem_name'],
                                                                                     likely = metsim_out[i]['output'][j]['precursor']['likelihood'])
                            metsim_cache.append(metsim_out[i]['output'][j]['precursor'])
                            print('Precursor query added to metadata cache...')
                        else:
                            print('Precursor SMILES found in cached results. Inserting into dictionary...')
                            metsim_out[i]['output'][j]['precursor'] = metsim_cache[[idx for idx in range(len(metsim_cache)) if metsim_cache[idx]['smiles'] == metsim_out[i]['output'][j]['precursor']['smiles']][0]]
                    else:
                        if metsim_out[i]['output'][j]['precursor']['smiles'] not in [cache_item['smiles'] for cache_item in metsim_cache]:
                            metsim_out[i]['output'][j]['precursor'] = metsim_hcd_out(smiles = metsim_out[i]['output'][j]['precursor']['smiles'],
                                                                                     casrn = metsim_out[i]['output'][j]['precursor']['casrn'],
                                                                                     dtxsid = metsim_out[i]['output'][j]['precursor']['dtxsid'],
                                                                                     chem_name = metsim_out[i]['output'][j]['precursor']['chem_name'])
                            metsim_cache.append(metsim_out[i]['output'][j]['precursor'])
                            print('Precursor query added to metadata cache...')
                        else:
                            print('Precursor SMILES found in cached results. Inserting into dictionary...')
                            metsim_out[i]['output'][j]['precursor'] = metsim_cache[[idx for idx in range(len(metsim_cache)) if metsim_cache[idx]['smiles'] == metsim_out[i]['output'][j]['precursor']['smiles']][0]]
                    for k in range(len(metsim_out[i]['output'][j]['successors'])): # k = number of metabolites per precursor
                        if 'likelihood' in list(metsim_out[i]['output'][j]['successors'][k]['metabolite'].keys()):
                            if metsim_out[i]['output'][j]['successors'][k]['metabolite']['smiles'] not in [cache_item['smiles'] for cache_item in metsim_cache]:
                                metsim_out[i]['output'][j]['successors'][k]['metabolite'] = metsim_hcd_out(smiles = metsim_out[i]['output'][j]['successors'][k]['metabolite']['smiles'],
                                                                                                           casrn = metsim_out[i]['output'][j]['successors'][k]['metabolite']['casrn'],
                                                                                                           dtxsid = metsim_out[i]['output'][j]['successors'][k]['metabolite']['dtxsid'],
                                                                                                           chem_name = metsim_out[i]['output'][j]['successors'][k]['metabolite']['chem_name'],
                                                                                                           likely = metsim_out[i]['output'][j]['successors'][k]['metabolite']['likelihood'])
                                metsim_cache.append(metsim_out[i]['output'][j]['successors'][k]['metabolite'])
                                print('Successor metabolite query added to metadata cache...')
                            else:
                                print('Successor metabolite SMILES found in cached results. Inserting into dictionary...')
                                metsim_out[i]['output'][j]['successors'][k]['metabolite'] = metsim_cache[[idx for idx in range(len(metsim_cache)) if metsim_cache[idx]['smiles'] == metsim_out[i]['output'][j]['successors'][k]['metabolite']['smiles']][0]] 
                        else:
                            if metsim_out[i]['output'][j]['successors'][k]['metabolite']['smiles'] not in [cache_item['smiles'] for cache_item in metsim_cache]:
                                metsim_out[i]['output'][j]['successors'][k]['metabolite'] = metsim_hcd_out(smiles = metsim_out[i]['output'][j]['successors'][k]['metabolite']['smiles'],
                                                                                                           casrn = metsim_out[i]['output'][j]['successors'][k]['metabolite']['casrn'],
                                                                                                           dtxsid = metsim_out[i]['output'][j]['successors'][k]['metabolite']['dtxsid'],
                                                                                                           chem_name = metsim_out[i]['output'][j]['successors'][k]['metabolite']['chem_name'])
                                metsim_cache.append(metsim_out[i]['output'][j]['successors'][k]['metabolite'])
                                print('Successor metabolite query added to metadata cache...')
                            else:
                                print('Successor metabolite SMILES found in cached results. Inserting into dictionary...')
                                metsim_out[i]['output'][j]['successors'][k]['metabolite'] = metsim_cache[[idx for idx in range(len(metsim_cache)) if metsim_cache[idx]['smiles'] == metsim_out[i]['output'][j]['successors'][k]['metabolite']['smiles']][0]] 
                        print('input: '+str(i+1)+'/'+str(len(metsim_out))+' precursor: '+str(j+1)+'/'+str(len(metsim_out[i]['output']))+' metabolite: '+str(k+1)+'/'+str(len(metsim_out[i]['output'][j]['successors'])))
                if fnam != None:
                    json.dump(metsim_out, open(fnam,'w'))
        else:
            return metsim_metadata_full(metsim_out = metsim_out, fnam = fnam, metsim_cache = [])
    else:
        raise('Please supply a metsim dataset (list of dictionaries)')
    # print(metsim_out)
    return metsim_out

In [5]:
#for generational tracking of output:
def recursive_gen_list(input_df = None,
                       parent_list = [],
                       successor_list = None,
                       out_list = [],
                       gen = 1):
    if type(input_df) == 'NoneType':
        raise('Please include BioTransformer output csv dataframe as input_df.')
    successor_dict = {}
    for i in parent_list.index:
        if pd.isna(parent_list[i]):
            successor_df = input_df.loc[pd.isna(input_df['Precursor ID']),:]
            successor_list = successor_df['Metabolite ID']
            print(str(len(successor_list))+' Successors found for parent chemical')
        else:
            successor_df = input_df.loc[input_df['Precursor ID'] == parent_list[i],:]
            successor_list = successor_df['Metabolite ID']
        if len(successor_list) > 0:

            successor_dict = {'precursor': {'smiles': successor_df['Precursor SMILES'].unique()[0],
                                        'inchikey': None,
                                        'casrn': None,
                                        'hcd_smiles': None,
                                        'dtxsid': None,
                                        'chem_name': None
                                       },
                          'successors': [{'enzyme': successor_df.loc[j,'Enzyme(s)'].split('\n'),
                                          'mechanism': successor_df.loc[j,'Reaction'],
                                          'generation': gen,
                                          'metabolite': {'smiles': successor_df.loc[j,'SMILES'],
                                                         'inchikey': None,
                                                         'casrn': None,
                                                         'hcd_smiles': None,
                                                         'dtxsid': None,
                                                         'chem_name': None
                                                        }
                                       } for j in successor_df.index]
                             }
            if pd.notna(parent_list[i]):
                print(str(len(successor_df))+' successors found for gen '+str(gen)+' precursor '+parent_list[i])
            if successor_dict['precursor']['smiles'] in [out_list[j]['precursor']['smiles'] for j in range(len(out_list))]:
                print('Precursor-successor relationship already recorded, skipping to next precursor.')
            else:
                out_list = out_list+[successor_dict]
                print('dict added to out_list for index '+str(i))
                #check for more generations:
                for k in successor_list.index:
                    print('starting recursion on successor index '+str(k))
                    out_list = recursive_gen_list(input_df = input_df,
                                                  parent_list = pd.Series(successor_list[k]),
                                                  successor_list = [],
                                                  out_list = out_list,
                                                  gen = gen+1)
                    print('recursion complete for successor index '+str(k))
            return out_list
        else:
            print('No successors for gen '+str(gen)+' precursor '+parent_list[i]+', moving to next gen 1 precursor.')
            gen = 1
            return out_list

In [6]:
# Function 3: BioTransformer 3.0 Human Phase I + Phase II Metsim.
def btrans_metsim_subprocess(models = ['cyp450','phaseII'],
                             cyp_mode = 1,
                             cycles = [2,1],
                             smiles = None,
                             casrn = None,
                             del_tmp = False,
                             dtxsid = None,
                             chem_name = None,
                             idx = None,
                             multi_proc = False):
    
    import pandas as pd
    import datetime
    import os
    import subprocess
    import tempfile
    
# Need to determine how to implement recursion with multiprocessing...
    
    #set up initial dictionary outputs:
    btrans_dict = {'datetime': str(datetime.datetime.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')),
                   'software': 'BioTransformer',
                   'version': 3.0,
                   'params':{'depth': sum([cycles[i] if ('allHuman' != models[i]) and ('superbio' != models[i]) else 4 for i in range(len(cycles))]),
                             'organism': 'Human',
                             'site_of_metabolism': False,
                             'model': [str(cycles[i])+'x '+models[i] for i in range(len(models))]
                            }
                  }
    btrans_dict['input'] = {'smiles': smiles,
                            'inchikey': None,
                            'casrn': casrn,
                            'hcd_smiles': None,
                            'dtxsid': dtxsid,
                            'chem_name': chem_name
                           } 
    
    #change directory to BioTransformer directory
    btrans_dir = 'C:/Users/lgroff/OneDrive - Environmental Protection Agency (EPA)/Profile/Documents/Data/GenRA/BioTransformer3.0'
    # btrans_dir = 'C:/Users/lgroff/OneDrive - Environmental Protection Agency (EPA)/Profile/Documents/Data/GenRA/BioTransformer1.1.5'
    if os.curdir != btrans_dir:
        os.chdir(btrans_dir)
       
    if pd.notna(smiles): #check that we have a SMILES string input
        #Begin constructing BioTransformer input command string:
        btrans_cmd = 'java -jar BioTransformer3.0_20220615.jar -k pred ' #current version
        # btrans_cmd = 'java -jar BioTransformer-1.1.5.jar -k pred ' #2019 paper version
        # btrans_cmd = 'java -jar BioTransformer-1-0-6.jar -k pred ' #original version (Only uses single models!)
        #incorporate model parameters into command string:
        if len(models) == 1:
            btrans_model = '-b "'+models[0]+'" '
            if cycles[0] > 1:
                btrans_model = btrans_model+'-s '+str(cycles[0])
        else:
            btrans_model = '-q "'
            for i in range(len(models)):
                if i < len(models)-1:
                    btrans_model = btrans_model+models[i]+':'+str(cycles[i])+';'
                else:
                    btrans_model = btrans_model+models[i]+':'+str(cycles[i])+'" '
        btrans_cmd = btrans_cmd+btrans_model+' -cm '+str(cyp_mode)+' ' #specify cyp_mode with later versions of BioTransformer
        # btrans_cmd = btrans_cmd+btrans_model+' ' #no cyp_mode in input command for original 2019 relase of BioTransformer
        
        #create temporary file to store BioTransformer output:
        print('Generating output tempfile for index '+str(idx)+', DTXSID: '+str(dtxsid)+'...')
        fnum, fnam = tempfile.mkstemp(dir = './tmpfiles',
                                      prefix = 'btrans_out_'+dtxsid+'_',
                                      suffix = '.csv')
        
        #Finish constructing input command using the tempfile name:
        fnam_split = fnam.split('\\')
        btrans_fnam = '.\\'+fnam_split[-2]+'\\'+fnam_split[-1]
        #insert input smiles string and output csv tempfile name:
        btrans_cmd = btrans_cmd + '-ismi "'+smiles+'" -ocsv "'+btrans_fnam+'"' #use Daylight SMILES input
        # btrans_cmd = btrans_cmd + '-isdf "'+smiles+'" -ocsv "'+btrans_fnam+'"'
        print(btrans_cmd)
        #Run BioTransformer with subprocess and the appropriate input command string:
        print('beginning metsim for index #'+str(idx)+'...')
        btrans_out = subprocess.run(btrans_cmd,
                                    shell=True,
                                    stdout=subprocess.PIPE) 
        
        #Preallocate data frame and read tempfile output if BioTransformer ran successfully:
        btrans_out_df = pd.DataFrame()
        if btrans_out.stderr == None:
            try:
                btrans_out_df = pd.read_csv(fnam)
            except:
                pass
        else:
            print('error, check stdout')
            print(btrans_out.stdout.decode())
            
        btrans_dict['output'] = []
        if multi_proc == False:
            #Process metabolism data (if it exists for the given smiles):
            if os.path.getsize(fnam) > 0:
                input_df = pd.read_csv(fnam)
                parent_list = input_df['Precursor ID']
                parent_list = parent_list.drop_duplicates()
                btrans_dict['output'] = recursive_gen_list(input_df = input_df,
                                                           parent_list = parent_list,
                                                           successor_list = [],
                                                           out_list = [],
                                                           gen = 1)
            else: #Valid SMILES given, no metabolites produced:
                print('No metabolites produced for index #'+str(i))
                btrans_dict['output'] = [{'precursor': btrans_dict['input'],
                                          'successors': [{'enzyme': [],
                                                          'mechanism': None,
                                                          'generation': None,
                                                          'metabolite': {'smiles': None,
                                                                         'inchikey': None,
                                                                         'casrn': None,
                                                                         'hcd_smiles': None,
                                                                         'dtxsid': None,
                                                                         'chem_name': None
                                                                        }
                                                        }]
                                        }]
        else:
            #Need to determine how to multiprocess recursion, until then, store input parameters and output structure.
            btrans_dict['output'] = [{'precursor': btrans_dict['input'],
                                      'successors': [{'enzyme': [],
                                                      'mechanism': None,
                                                      'generation': None,
                                                      'metabolite': {'smiles': None,
                                                                     'inchikey': None,
                                                                     'casrn': None,
                                                                     'hcd_smiles': None,
                                                                     'dtxsid': None,
                                                                     'chem_name': None
                                                                    }
                                                    }]
                                    }]
        #close temporary output file. Will delete if del_tempfile function input is set to True.
        os.close(fnum)
        if del_tmp == True:
            os.remove(fnam)
    else:
        print('No SMILES string provided for index #'+str(idx)+".")
        #This list returns if no metabolites are formed/BioTransformer fails.
        btrans_dict['output'] = [{'precursor': {'smiles': None,
                                                'inchikey': None,
                                                'casrn': None,
                                                'hcd_smiles': None,
                                                'dtxsid': None,
                                                'chem_name': None
                                               },
                                  'successors': [{'enzyme': [],
                                                  'mechanism': None,
                                                  'generation': None,
                                                  'metabolite': {'smiles': None,
                                                                 'inchikey': None,
                                                                 'casrn': None,
                                                                 'hcd_smiles': None,
                                                                 'dtxsid': None,
                                                                 'chem_name': None
                                                                }
                                                }]
                                }]
        btrans_dict['input'] = {'smiles': None,
                                'inchikey': None,
                                'casrn': None,
                                'hcd_smiles': None,
                                'dtxsid': None,
                                'chem_name': None
                               }
    print('MetSim complete for index '+str(idx)+'.')
    if del_tmp == False:
        #until figuring out how to implement recursive metabolite search with multiprocessing, return filename for separate analysis function.
        return (idx, btrans_dict, fnam)
    else:
        return (idx, btrans_dict, None)

In [ ]:
# # Load in existing dictionary instead of rerunning.
#github URL for RAW readout of SMPDB_59Parents.json (update token in URL as necessary)
smpdb_obs_pathways = json.loads(open('smpdb_jcim_valid_aggregate_112parents.json','r'))

out_list = []
pool = mp.Pool(mp.cpu_count()) #define number of available CPU cores for multiprocessing.

out_list = pool.starmap_async(btrans_metsim_subprocess,
                              #arguments (must be listed in the same order as given in the function definition):
                              [(['ecbased','cyp450','phaseII'], #models
                                 1, #cyp_mode
                                 [1,2,1], #cycles
                                 smpdb_obs_pathways[idx]['input']['hcd_smiles'], #smiles
                                 smpdb_obs_pathways[idx]['input']['casrn'], #casrn
                                 False, #del_tmp
                                 smpdb_obs_pathways[idx]['input']['dtxsid'], #dtxsid
                                 smpdb_obs_pathways[idx]['input']['chem_name'], #chem_name
                                 idx, #index
                                 True #multi_proc
                               )
                               for idx in range(len(smpdb_obs_pathways[0:4]))]).get() #show for first four chemicals in SMPDB dataset as a test

pool.close() #close the processing pool to release resources.

# Post-processing of multiprocessed BioTransformer Data
for i in range(len(out_list)):
    if os.path.getsize(out_list[i][2]) > 0:
        input_df = pd.read_csv(out_list[i][2])
        parent_list = input_df['Precursor ID']
        parent_list = parent_list.drop_duplicates()
        out_list[i][1]['output'] = recursive_gen_list(input_df = input_df,
                                                      parent_list = parent_list,
                                                      successor_list = [],
                                                      out_list = [],
                                                      gen = 1)
    else: #Valid SMILES given, no metabolites produced:
            print('No metabolites produced for index #'+str(i))
            out_list[i][1]['output'] = [{'precursor': out_list[i][1]['input'],
                                         'successors': [{'enzyme': [],
                                                         'mechanism': None,
                                                         'generation': None,
                                                         'metabolite': {'smiles': None,
                                                                        'inchikey': None,
                                                                        'casrn': None,
                                                                        'hcd_smiles': None,
                                                                        'dtxsid': None,
                                                                        'chem_name': None
                                                                       }
                                                       }]
                                       }]
preds_complete = [out_list[i][1] for i in range(len(out_list))]
json.dump(preds_complete,open('metsim_biotransformer_1xecbased_2xcyp450_1xphase2_smpdb_test.json','w'))

In [34]:
preds_complete_all_metadata = metsim_metadata_full(preds_complete_all_metadata,fnam = 'metsim_biotransformer_1xecbased_2xcyp450_1xphase2_smpdb_test.json')